<h3>Глобальные переменные</h3>

In [1]:
#путь до языковой модели, которая будет учиться в составе сетки. Полный список в https://huggingface.co/models
transformer_model_path = './bert-base-cased'

#Использовать ли видеокарты. torch может автоматически определить эту переменную
#Для этого расскоментируйте строки
#import torch
#use_cuda = torch.cuda.is_available()
use_cuda = False

<h3>Векторизуем словарь Meddra</h3>

In [2]:
from vectorization import ConceptVectorizer

In [6]:
import torch
torch.cuda.is_available()

True

In [4]:
#сюда прописывается путь до модели и путь до файла словаря расширения .asc, который необходимо векторизовать
CV = ConceptVectorizer(transformer_model_path, '../../Data/External/pt_en.asc', \
                             use_concept_less=False, use_cuda=use_cuda)

Some weights of the model checkpoint at ./bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Расчет вложений словаря

In [5]:
#используется только для векторизации словаря
#когда вектора готовы, проще их загрузить ячейкой с загрузкой
CV.fit_transform(mode='mean_pooling')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


getting concept embeddings in mean_pooling mode...
Compute embeddings...
in cpu mode there is no progress bar
Embedding aggregation...
Concept embeddings have computed in 251.15103459358215 seconds


Сохранение рассчитанных векторов концептов

In [5]:
import torch

torch.save(CV.thesaurus_embeddings, 'model_thesaurus_embeddings.pt')

Загрузка готовых вложений (если они есть). Загружать вложения после fit_transform() не надо

In [ ]:
import torch

CV.thesaurus_embeddings = torch.load('model_thesaurus_embeddings.pt')
CV.normalization_mode = 'mean_pooling'

<h3>Обработка датасета, пример с CADEC</h3>

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

data_dir = '../../Data/Raw/cadec_table_grartem.csv'

df = pd.read_csv(data_dir)
df

,docName,term,meddra
0,LIPITOR.877,MUSCLES ACHED,10028411
1,LIPITOR.877,DIFFICULTY IN WALKING,10017577
2,LIPITOR.877,AFFECTED MY BALANCE,10027175
3,LIPITOR.169,gas,10016766
4,LIPITOR.169,loose stools,10012735
...,...,...,...
6313,LIPITOR.346,tremendous neck pain,10028836
6314,LIPITOR.346,severe headaches,10019211
6315,LIPITOR.346,rash,10037844
6316,LIPITOR.346,hemorrhoids,10019022


Поле **term** в _df_ содержит фразу, которую необходимо нормализовать<br>
Поле **meddra** в _df_ код концепта в словаре meddra<br>
<br>
В случае, когда у фразы несколько кодов meddra, они записаны через символ '|' в колонке **meddra**, например '100105|100106'<br>
В случае, когда у фразы НЕТ концепта, в колонке **meddra** у нее стоит 'CONCEPT_LESS'

<h4>Тут нужно дописать часть обработки данных</h4>

а именно:
- Отфильтровать строки со значением CONCEPT_LESS
- У всех фраз, у которых несколько кодов meddra, оставить только один код
- Отфильтровать фразы у которых кода концепта нет в списке CV.meddra_codes
- Разделить набор данных на train и test часть. Фразы в списках train_phrases / test_phrases, соотвествующие им концепты в списках train_concepts / test_concepts

In [ ]:
#ячейка с кодом обработки

In [10]:
from dataset import MedNormDataset

ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=use_cuda)
ds_test = MedNormDataset(test_phrases, test_concepts, CV, use_cuda=use_cuda)

<h2>Импорт модели</h2>

In [11]:
from models import CADEC_SoTa


net = CADEC_SoTa(transformer_model_path, CV.thesaurus_embeddings)
#перемещение модели на вычислительные мощности, либо cpu, либо cuda
device = 'cuda' if use_cuda else 'cpu'
net.to(device)
print('Net loaded')

Some weights of the model checkpoint at ./bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Net loaded


<h3>Инициализация доп. объектов для обучения</h3>

In [12]:
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.0001)

In [13]:
scaler = torch.cuda.amp.GradScaler()

Настройка глобальных переменных для большей детерменированности <br>
Это нужно, чтобы одна и та же сетка сходилась к одним и тем же значениям при одних и тех же параметрах для обучения

In [15]:
import os 

os.environ["CUBLAS_WORKSPACE_CONFIG"]=":16:8"
torch.use_deterministic_algorithms(mode=False)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False

<h3>Процесс обучения и рассчета точностей</h3>

In [16]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score

#Гиперпараметры
batch_size=16
epochs = 1
    
trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)

net.train()
initial_loss = None
for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if use_cuda:
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    real_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'])
    
    print(f1_score(real_answers, model_answers, average='micro'))
            
print('Finished Training')

100%|██████████| 1970/1970 [01:45<00:00, 18.70batch/s]

0.5106598984771573
Finished Training


Инференс (подать фразу в сетку и посмотреть, какой концепт она выдаст)

Чтобы сетка выдавала отсутствие концепта на несвязанные с медрой фразы, раскомментируйте строчку с label_concepless_tensors

In [17]:
from transformers import AutoTokenizer

phrase = 'Disease'
tokenizer = AutoTokenizer.from_pretrained(transformer_model_path)
encoded_input = tokenizer([phrase], padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    outputs_dict = net(inputs)
    #outputs_dict.label_concepless_tensors(score_treshold = 6.1977e-05)
    pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]
    
print('phrase: %s'%phrase)
print('model output: %s'%CV.meddra_code_to_meddra_term[pred_meddra_code])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


phrase: Disease
model output: Physical disability


<h2>Сохранение и загрузка модели</h2>

сохранение

In [63]:
torch.save(net, './trained_model_weights.pt')
torch.save(optimizer.state_dict(), './trained_model_opt.pt')

загрузка

In [64]:
the_model = torch.load('./trained_model_weights.pt')